<a href="https://colab.research.google.com/github/CoffeBank/ML_work_6/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import csv

tsv_file = "/content/drive/MyDrive/Colab Notebooks/pairs.tsv"
training_data = []
original = []
translation = []

with open(tsv_file, 'r', encoding='utf-8') as file:
    tsv_reader = csv.reader(file, delimiter='\t')
    cnt = 0
    for row in tsv_reader:
        training_data.append((row[1], row[3]))
        cnt += 1
        if cnt == 700000:
            break

source_vocab = set()
target_vocab = set()
for source, target in training_data:
    source_vocab.update(source.split())
    target_vocab.update(target.split())

source_vocab = sorted(list(source_vocab))
target_vocab = sorted(list(target_vocab))

source_vocab = ['<SOS>', '<EOS>'] + sorted(list(source_vocab))
target_vocab = ['<SOS>', '<EOS>'] + sorted(list(target_vocab))

source_word2index = {word: i for i, word in enumerate(source_vocab)}
source_index2word = {i: word for word, i in enumerate(source_vocab)}

target_word2index = {word: i for i, word in enumerate(target_vocab)}
target_index2word = {i: word for word, i in enumerate(target_vocab)}

source_vocab_size = len(source_vocab)
target_vocab_size = len(target_vocab)

source_word2index = {word: i for i, word in enumerate(source_vocab)}
source_index2word = {i: word for word, i in source_word2index.items()}

target_word2index = {word: i for i, word in enumerate(target_vocab)}
target_index2word = {i: word for word, i in target_word2index.items()}

def encode_sequence(sequence, word2index):
    return [word2index[word] for word in sequence.split()]

X_train = [encode_sequence(data[0], source_word2index) for data in training_data]
Y_train = [encode_sequence(data[1], target_word2index) for data in training_data]

class Seq2Seq(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Seq2Seq, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        output = self.out(output[0])
        return output, hidden

    def init_hidden(self, device):
        return torch.zeros(1, 1, self.hidden_size, device=device)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

hidden_size = 256
encoder = Seq2Seq(source_vocab_size, hidden_size, hidden_size).to(device)
decoder = Seq2Seq(target_vocab_size, hidden_size, target_vocab_size).to(device)

cuda


In [27]:

criterion = nn.CrossEntropyLoss()
encoder_optimizer = optim.SGD(encoder.parameters(), lr=0.01)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=0.01)

n_epochs = 1
for epoch in range(n_epochs):
    total_loss = 0
    for i in range(len(X_train)):
        encoder_hidden = encoder.init_hidden(device)

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        source_tensor = torch.tensor(X_train[i], dtype=torch.long).to(device)
        target_tensor = torch.tensor(Y_train[i], dtype=torch.long).to(device)

        source_length = source_tensor.size(0)
        target_length = target_tensor.size(0)

        loss = 0

        for ei in range(source_length):
            _, encoder_hidden = encoder(source_tensor[ei], encoder_hidden)

        decoder_input = torch.tensor([[target_word2index['<SOS>']]])

        decoder_hidden = encoder_hidden

        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input.to(device), decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()

            loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))

        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item() / target_length

    #if epoch % 10 == 0:
    print('Epoch %d, Loss=%.4f' % (epoch, total_loss / len(X_train)))

encoder_path = "/content/drive/MyDrive/Colab Notebooks/encoder_model.pth"
decoder_path = "/content/drive/MyDrive/Colab Notebooks/decoder_model.pth"

torch.save(encoder.state_dict(), encoder_path)
torch.save(decoder.state_dict(), decoder_path)

print("Модели сохранены")

KeyboardInterrupt: 

In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import csv

encoder_checkpoint = "/content/drive/MyDrive/Colab Notebooks/encoder_model.pth"
decoder_checkpoint = "/content/drive/MyDrive/Colab Notebooks/decoder_model.pth"

encoder_state_dict = torch.load(encoder_checkpoint)
decoder_state_dict = torch.load(decoder_checkpoint)

encoder.load_state_dict(encoder_state_dict)
decoder.load_state_dict(decoder_state_dict)

def translate_sentence(sentence):
    with torch.no_grad():
        input_tensor = torch.tensor(encode_sequence(sentence, source_word2index), dtype=torch.long).to(device)
        encoder_hidden = encoder.init_hidden(device)

        source_length = input_tensor.size(0)
        for ei in range(source_length):
            _, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)

        decoder_input = torch.tensor([[target_word2index['<SOS>']]]).to(device)
        decoder_hidden = encoder_hidden

        decoded_words = []
        for di in range(10):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            if topi.item() == target_word2index['<EOS>']:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(target_index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return ' '.join(decoded_words)


sentence_to_translate = "Что ты любишь делать"
translated_sentence = translate_sentence(sentence_to_translate)
print("Переведенное предложение:", translated_sentence)


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.